<h1 style='font-weight:bold; text-align: justify'> Test </h1>
<h1 style='font-weight:bold; text-align: justify'> Machine Learning STQD6024 </h1>
<h1 style='font-weight:bold; text-align: justify'> Hazim Fitri Bin Ahmad Faudzi P152419 </h1>

<h1 style='font-weight:bold; color:orange;'> a) Model building </h1>

In [20]:
# import necessary library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector

import statsmodels.api as sm
import time
import itertools

In [21]:
# read data
df = pd.read_table('Question_2.txt', sep=',')
df

,No,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP(cm),FLOW(cm),Compressive Strength (28-day)(Mpa)
0,1,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0,62.0,34.99
1,2,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0,20.0,41.14
2,3,162.0,148.0,191.0,179.0,16.0,840.0,743.0,1.0,20.0,41.81
3,4,162.0,148.0,190.0,179.0,19.0,838.0,741.0,3.0,21.5,42.08
4,5,154.0,112.0,144.0,220.0,10.0,923.0,658.0,20.0,64.0,26.82
...,...,...,...,...,...,...,...,...,...,...,...
95,96,184.3,153.4,239.2,179.0,7.5,920.2,640.9,0.0,20.0,41.01
96,97,215.6,112.9,239.0,198.7,7.4,884.0,649.1,27.5,64.0,39.13
97,98,295.3,0.0,239.9,236.2,8.3,780.3,722.9,25.0,77.0,44.08
98,99,248.3,101.0,239.1,168.9,7.7,954.2,640.6,0.0,20.0,49.97


In [22]:
# define response (Y) and predictor (X) variables
y = df['Compressive Strength (28-day)(Mpa)']
x = df.drop(['Compressive Strength (28-day)(Mpa)', 'SLUMP(cm)', 'FLOW(cm)', 'No'], axis = 1)

In [23]:
# split data intro training set and testing set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [24]:
# check variables data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   No                                  100 non-null    int64  
 1   Cement                              100 non-null    float64
 2   Slag                                100 non-null    float64
 3   Fly ash                             100 non-null    float64
 4   Water                               100 non-null    float64
 5   SP                                  100 non-null    float64
 6   Coarse Aggr.                        100 non-null    float64
 7   Fine Aggr.                          100 non-null    float64
 8   SLUMP(cm)                           100 non-null    float64
 9   FLOW(cm)                            100 non-null    float64
 10  Compressive Strength (28-day)(Mpa)  100 non-null    float64
dtypes: float64(10), int64(1)
memory usage: 8.7 KB


In [25]:
scale = StandardScaler()

x_train_scale = scale.fit_transform(x_train)
x_test_scale = scale.transform(x_test)

In [26]:
lr = LinearRegression()

lr.fit(x_train_scale, y_train)

y_predict = lr.predict(x_train_scale)

y_predict_test = lr.predict(x_test_scale)

col_name = x.columns

print("Regression model coefficients:")
print(pd.DataFrame( {'Variables':col_name, 'Coefficient':lr.coef_}))
print('============================\n')
print(f"Regression model intercept:{lr.intercept_:.4f}")

Regression model coefficients:
      Variables  Coefficient
0        Cement     4.898330
1          Slag    -1.755212
2       Fly ash     4.658713
3         Water    -4.545214
4            SP     0.423314
5  Coarse Aggr.    -4.638413
6    Fine Aggr.    -2.257704

Regression model intercept:35.8230


<h1 style='font-weight:bold; color:orange;'> b) Compute MSE on test set </h1>

In [27]:
print(f"MSE for training set: {mean_squared_error(y_train, y_predict):.4f}")
print(f"MSE for testing set: {mean_squared_error(y_test, y_predict_test):.4f}")

MSE for training set: 6.8980
MSE for testing set: 4.0327


<h1 style='font-weight:bold; color:orange;'> c) Optimal model using forward selection </h1>

In [29]:

sfs = SequentialFeatureSelector(lr, scoring='neg_mean_squared_error')

sfs.fit(x_train_scale, y_train)

selected_features = sfs.get_feature_names_out(input_features=x.columns)
print(selected_features)

['Cement' 'Fly ash' 'Water']


In [30]:
selected_mask = sfs.get_support()
selected_f = x.columns[selected_mask]
print("Selected Feature", list(selected_f))

Selected Feature ['Cement', 'Fly ash', 'Water']


In [35]:
?SFS

Init signature:
SFS(
    estimator,
    k_features=1,
    forward=True,
    floating=False,
    verbose=0,
    scoring=None,
    cv=5,
    n_jobs=1,
    pre_dispatch='2*n_jobs',
    clone_estimator=True,
    fixed_features=None,
    feature_groups=None,
)
Docstring:     
Sequential Feature Selection for Classification and Regression.

Parameters
----------
estimator : scikit-learn classifier or regressor
k_features : int or tuple or str (default: 1)
    Number of features to select,
    where k_features < the full feature set.
    New in 0.4.2: A tuple containing a min and max value can be provided,
        and the SFS will consider return any feature combination between
        min and max that scored highest in cross-validation. For example,
        the tuple (1, 4) will return any combination from
        1 up to 4 features instead of a fixed number of features k.
    New in 0.8.0: A string argument "best" or "parsimonious".
        If "best" is provided, the feature selector will r

In [37]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression

sfs = SFS(lr,
          k_features='best',
          forward=True,
          floating=False,
          verbose=2, 
          scoring='neg_mean_squared_error',
          cv=5)

sfs.fit(x_train_scale, y_train)

[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s finished

[2025-07-13 05:32:49] Features: 1/7 -- score: -48.1942858083583[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished

[2025-07-13 05:32:49] Features: 2/7 -- score: -16.025213466843518[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished

[2025-07-13 05:32:49] Features: 3/7 -- score: -11.951366211333275[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished

[2025-07-13 05:32:49] Features: 4/7 -- score: -9.173065559468114[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2025-07-13 05:32:50] Features: 5/7 -- score: -8.793053747264494[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2025-07-13 05:32:50] Features: 6/7 -- score: -8.692217351541895[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2025-07-13 05:32:50] Features: 7/7 -- score: -8.835889035421346

,estimator,LinearRegression()
,k_features,"(1, ...)"
,forward,True
,floating,False
,verbose,2
,scoring,'neg_mean_squared_error'
,cv,5
,n_jobs,1
,pre_dispatch,'2*n_jobs'
,clone_estimator,True
,fixed_features,None


In [57]:
forward_selection_index = x.columns[list(sfs.k_feature_idx_)]

x_train_selected = pd.DataFrame(x_train_scale, columns=x.columns)[list(forward_selection_index)]
x_train_selected

,Cement,Slag,Fly ash,Water,Coarse Aggr.,Fine Aggr.
0,-0.862031,1.820472,-1.691506,0.501832,-0.711184,1.274604
1,-0.971217,0.477080,1.039973,-1.412651,1.355576,-1.132575
2,1.029683,0.478716,-0.073358,0.933670,-1.149448,-0.832476
3,-0.849335,1.051417,0.450831,0.261922,-0.618918,-0.449371
4,1.385173,0.855062,-1.691506,1.173580,-1.460846,0.779759
...,...,...,...,...,...,...
75,0.648801,-1.337563,-0.358243,0.357886,-0.088388,0.971312
76,1.093163,0.495079,-1.691506,1.077616,-1.022582,0.811685
77,1.867623,-1.337563,-1.691506,-0.361844,1.503202,-0.130116
78,0.976359,-1.334290,1.041113,1.864521,-1.182895,-0.364768


In [67]:
model = LinearRegression()
model.fit(x_train_selected, y_train)
model.predict(new_df.drop('SP', axis=1))

array([-7782.75179279, -6850.42568748])

In [34]:
for k in range(1, len(sfs.subsets_) + 1):
    subset = sfs.subsets_[k]
    feature_indices = subset['feature_idx']
    feature_names = x.columns[list(feature_indices)]
    score = subset['avg_score']
    print(f"Step {k}: Features {list(feature_names)} → Score: {score:.4f}")


Step 1: Features ['Fly ash'] → Score: -48.1943
Step 2: Features ['Cement', 'Fly ash'] → Score: -16.0252
Step 3: Features ['Cement', 'Fly ash', 'Water'] → Score: -11.9514
Step 4: Features ['Cement', 'Fly ash', 'Water', 'Coarse Aggr.'] → Score: -9.1731
Step 5: Features ['Cement', 'Fly ash', 'Water', 'Coarse Aggr.', 'Fine Aggr.'] → Score: -8.7931
Step 6: Features ['Cement', 'Slag', 'Fly ash', 'Water', 'Coarse Aggr.', 'Fine Aggr.'] → Score: -8.6922
Step 7: Features ['Cement', 'Slag', 'Fly ash', 'Water', 'SP', 'Coarse Aggr.', 'Fine Aggr.'] → Score: -8.8359


<h1 style='font-weight:bold; color:orange;'> d) Model prediction </h1>

In [ ]:
x_new = ['Fine Aggr.', 'Fly ash', 'Cement', 'Water', 'Slag', 'SP']

df.drop(['Compressive Strength (28-day)(Mpa)', 'SLUMP(cm)', 'FLOW(cm)', 'No', Coarse Aggr.])

In [65]:
new_df = pd.DataFrame({'Cement':[258.8,348.7], 'Slag':[88,0.1], 'Fly ash':[239.6, 223.1], 'Water':[175.3, 208.5], 'SP':[7.6, 9.6], 'Coarse Aggr.':[938.9, 786.2], 'Fine Aggr.':[646, 758.1]})
new_df

,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.
0,258.8,88.0,239.6,175.3,7.6,938.9,646.0
1,348.7,0.1,223.1,208.5,9.6,786.2,758.1


In [ ]:
model.predict(new_df)

c:\Users\PC 13\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


array([-4341.76193057, -3518.84509646])